## Note :
<h4>1. Pengembangan klasifikasi waktu sebagai berikut :</h4>

- Pagi (06.00 - 10.59)
- Siang (11.00 - 14.59)
- Sore (15.00 - 17.59)
- Malam (18.00 - 23.59)
- Dini Hari (00.00 - 03.59)
- Subuh (04.00 - 05.59)

In [8]:
import requests
import m3u8
import urllib3
import os
import glob
from datetime import datetime
import time
import cv2

from moviepy.editor import VideoFileClip, concatenate_videoclips
import subprocess

# Disable insecure request warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [9]:
def strip_end(text, suffix):
    if not text.endswith(suffix):
        return text
    return text[:len(text)-len(suffix)]

def download_file(url):
    local_filename = url.split('/')[-1]
    print(f'link download ts file: {url}')
    
    while True:
        response = requests.get(url, stream=True, verify=False)
        response.raise_for_status()
        
        # write response content to a file
        with open("video.ts", "wb") as f:
            f.write(response.content)
        
        # check the size of the downloaded file
        file_size = os.path.getsize("video.ts")
        if file_size < 10 * 1024:  # if file size is less than 10 KB
            print(f"Downloaded file is only {file_size} bytes. Retrying...")
            os.remove("video.ts")  # delete the previous downloaded file
            continue  # retry the download
        else:
            print(f"Downloaded file size: {file_size} bytes")
            break  # file size is OK, exit the loop
        
    cur_path = os.getcwd()
    ts_loc = cur_path + '/ts_files'
    
    if not os.path.exists(f'{ts_loc}'):
        os.makedirs(f'{ts_loc}')
        
    f = open(f"{ts_loc}/{local_filename}", "wb")
    f.write(response.content)
    
    return local_filename

def mp4_conversion(file_path):
    infile = file_path
    outfile = file_path.replace('.ts','.mp4')
    
    pre_name = os.path.basename(file_path)
    pos_name = pre_name.replace('.ts','.mp4')
    
    clip = VideoFileClip(infile)
    clip.write_videofile(outfile, fps=20)
    clip.close()
    
    del clip
    
    print(f'######### {pre_name} converted into {pos_name} #########')

In [10]:
m3u8_urls = [
    {'url': "https://atcs-dishub.bandung.go.id:1990/Kopo/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Kopo/' ,'region' : "SP Soekarno Hatta - Kopo", 'name': 'Kopo'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/GedebageUtara/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/GedebageUtara/' ,'region' : "SP Soekarno Hatta - Gedebage", 'name': 'Gedebage Arah Utara'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/GedebageSel/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/GedebageSelatan/' ,'region' : "SP Soekarno Hatta - Gedebage", 'name': 'Gedebage Arah Selatan'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/Buahbatu/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Buahbatu/' ,'region' : "SP Soekarno Hatta - Buahbatu", 'name': 'Buahbatu'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/Batununggal/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Batununggal/' ,'region' : "SP Soekarno Hatta - Batununggal", 'name': 'Batununggal'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/MochToha/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/MochToha/' ,'region' : "SP Soekarno Hatta - Moch. Toha", 'name': 'Moch. Toha'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/IstanaPlaza/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/IstanaPlaza/' ,'region' : "PTZ SP. Istana Plaza", 'name': 'Istana Plaza'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/CihampelasUtara/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/CihampelasUtara/' ,'region' : "PTZ SP. Istana Plaza", 'name': 'Istana Plaza'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/laswiut/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/laswiut/' ,'region' : "Laswi", 'name': 'Jalan Laswi'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/OtistaLingkar/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/OtistaLingkar/' ,'region' : "Otista", 'name': 'Otista Lingkar'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/PtInti/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/PtInti/' ,'region' : "PT Inti", 'name': 'PT Inti'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/MRamdan/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/MRamdan/' ,'region' : "M Ramdan", 'name': 'M Ramdan'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/RamdanBarat/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/RamdanBarat/' ,'region' : "M Ramdan", 'name': 'M Ramdan Barat'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/Sriwijaya/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Sriwijaya/' ,'region' : "Sriwijaya", 'name': 'Sriwijaya'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/BuahBatuLingkar/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/BuahBatuLingkar/' ,'region' : "Buah Batu Lingkar", 'name': 'Lingkar Buah Batu'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/Martanegara/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/Martanegara/' ,'region' : "Martanegara", 'name': 'Martanegara'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/TalagaBodas/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/TalagaBodas/' ,'region' : "Talaga Bodas", 'name': 'Talaga Bodas'}
    , {'url': "https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/stream.m3u8", 'ts_path': 'https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/' ,'region' : "Gatsu Lingkar", 'name': 'Gatsu Lingkar'}
]

In [11]:
waktu_now = datetime.today().strftime("%Y-%m-%d-%H")
jam_kejadian = int(waktu_now[-2:])

In [12]:
for link in m3u8_urls[-1:]:
    print(f"########## Akan download cctv daerah : {link['name']}")
    
    url_link = link['url']
    
    max_retries = 3
    retry_wait_time = 5
    
    for trial in range(max_retries):
        try:
            r = requests.get(url_link, verify=False)
            if r.status_code == 404:
                print(f"########## WARNING Daerah : {link['name']} tidak bisa diakses")
                continue
            m3u8_master = m3u8.loads(r.text)

            # print statement can be deleted, they were placed prior to debugging purposes.
            ts_path = link['ts_path']
            for seg in m3u8_master.data['segments']:
                url = ts_path + seg['uri']
                print(f'downloading {seg["uri"]}')
                download_file(url)  #download_file is a self made function
                
        except :
            print(f"HTTP error occurred, retrying in {retry_wait_time} seconds...")
            time.sleep(retry_wait_time)
    
    cur_path = os.getcwd()
    ts_loc = cur_path + '/ts_files'
    
    ts_files = glob.glob(f"{ts_loc}/*.ts")
    ts_files.sort()
    
    for ts in ts_files:
        try:
            mp4_conversion(ts)
            print('berhasil konversi ke mp4')
        except:
            print(f'cannot convert {os.path.basename(ts)}')
    
    mp4_files = glob.glob(f"{ts_loc}/*.mp4")
    mp4_files.sort()
    
    wadah_video = []
    for vid in mp4_files:
        func_vid = VideoFileClip(vid)
        wadah_video += [func_vid]

    # Concat them
    try:
        final = concatenate_videoclips(wadah_video)
        daerah = link['name']
        final_name = f'{daerah}_{waktu_now}.mp4'
    except:
        print(f"Gagal dapat CCTV {link['name']}")
        continue
    
    # Write output to the file
    output_path = f"{cur_path}/output_file"
    namadaerah = f"{link['name']}"
    folder_daerah = f"{output_path}/{namadaerah}"
    
    if not os.path.exists(f'{folder_daerah}'):
        os.makedirs(f'{folder_daerah}')
    
    print(f'membuat file : {final_name}')
    final.write_videofile(f'{folder_daerah}/{final_name}')
    
    for vid in mp4_files:
        try:
            os.remove(f"{vid}")
        except:
            print(f'cannot delete yet')
    
    for ts in glob.glob(f"{ts_loc}/*.ts"):
        try:
            os.unlink(ts)
            os.remove(f"{ts}")
            print('berhasil hapus ts file')
        except:
            pass

########## Akan download cctv daerah : Gatsu Lingkar
downloading seg355.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/seg355.ts
Downloaded file size: 96256 bytes
downloading seg356.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/seg356.ts
Downloaded file size: 96068 bytes
downloading seg357.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/seg357.ts
Downloaded file size: 128028 bytes
downloading seg358.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/seg358.ts
Downloaded file size: 157732 bytes
downloading seg359.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/seg359.ts
Downloaded file size: 154536 bytes
downloading seg360.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/seg360.ts
Downloaded file size: 150400 bytes
downloading seg361.ts
link download ts file: https://atcs-dishub.bandung.go.id:1990/GatsuLingkar/

Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping/ts_files\seg355.mp4
######### seg355.ts converted into seg355.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping/ts_files\seg356.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping/ts_files\seg356.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping/ts_files\seg356.mp4
######### seg356.ts converted into seg356.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping/ts_files\seg357.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping/ts_files\seg357.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping/ts_files\seg357.mp4
######### seg357.ts converted into seg357.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping/ts_files\seg358.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping/ts_files\seg358.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping/ts_files\seg358.mp4
######### seg358.ts converted into seg358.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping/ts_files\seg359.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping/ts_files\seg359.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping/ts_files\seg359.mp4
######### seg359.ts converted into seg359.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping/ts_files\seg360.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping/ts_files\seg360.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping/ts_files\seg360.mp4
######### seg360.ts converted into seg360.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping/ts_files\seg361.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping/ts_files\seg361.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping/ts_files\seg361.mp4
######### seg361.ts converted into seg361.mp4 #########
berhasil konversi ke mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping/ts_files\seg362.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping/ts_files\seg362.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping/ts_files\seg362.mp4
######### seg362.ts converted into seg362.mp4 #########
berhasil konversi ke mp4
membuat file : Gatsu Lingkar_2023-05-09-14.mp4
Moviepy - Building video C:\Users\GE00000668\Documents\Scraping/output_file/Gatsu Lingkar/Gatsu Lingkar_2023-05-09-14.mp4.
Moviepy - Writing video C:\Users\GE00000668\Documents\Scraping/output_file/Gatsu Lingkar/Gatsu Lingkar_2023-05-09-14.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\GE00000668\Documents\Scraping/output_file/Gatsu Lingkar/Gatsu Lingkar_2023-05-09-14.mp4


In [6]:
video_path = f'{folder_daerah}/{final_name}'
output_pic = f"{cur_path}/output_pic"
folder_daerah_gambar = f"{output_pic}/{daerah}"

if not os.path.exists(f'{folder_daerah_gambar}'):
    os.makedirs(f'{folder_daerah_gambar}')
    os.makedirs(f'{folder_daerah_gambar}/1. Pagi')
    os.makedirs(f'{folder_daerah_gambar}/2. Siang')
    os.makedirs(f'{folder_daerah_gambar}/3. Sore')
    os.makedirs(f'{folder_daerah_gambar}/4. Malam')
    os.makedirs(f'{folder_daerah_gambar}/5. Dini Hari')
    os.makedirs(f'{folder_daerah_gambar}/6. Subuh')

vidcap = cv2.VideoCapture(video_path)

In [7]:
def getFrame(sec, count):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        if 6 <= jam_kejadian < 10:
            cv2.imwrite(f"{folder_daerah_gambar}/1. Pagi/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        elif 11 <= jam_kejadian < 15:
            cv2.imwrite(f"{folder_daerah_gambar}/2. Siang/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        elif 15 <= jam_kejadian < 18:
            cv2.imwrite(f"{folder_daerah_gambar}/3. Sore/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        elif 18 <= jam_kejadian < 24:
            cv2.imwrite(f"{folder_daerah_gambar}/4. Malam/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        elif 0 <= jam_kejadian < 4:
            cv2.imwrite(f"{folder_daerah_gambar}/5. Dini Hari/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
        else :
            cv2.imwrite(f"{folder_daerah_gambar}/6. Subuh/{daerah.lower()}_{waktu_now}_{count}.jpg",image)
            
    return hasFrames

sec = 0
frameRate = 1 #//it will capture image in each 1 second

count = 1

if 6 <= jam_kejadian < 10:
    while os.path.exists(f"{folder_daerah_gambar}/1. Pagi/{daerah.lower()}_{waktu_now}_{count}.jpg"):
        count += 1
elif 11 <= jam_kejadian < 15:
    while os.path.exists(f"{folder_daerah_gambar}/2. Siang/{daerah.lower()}_{waktu_now}_{count}.jpg"):
        count += 1
elif 15 <= jam_kejadian < 18:
    while os.path.exists(f"{folder_daerah_gambar}/3. Sore/{daerah.lower()}_{waktu_now}_{count}.jpg"):
        count += 1
elif 18 <= jam_kejadian < 24:
    while os.path.exists(f"{folder_daerah_gambar}/4. Malam/{daerah.lower()}_{waktu_now}_{count}.jpg"):
        count += 1
elif 0 <= jam_kejadian < 4:
    while os.path.exists(f"{folder_daerah_gambar}/5. Dini Hari/{daerah.lower()}_{waktu_now}_{count}.jpg"):
        count += 1
else :
    while os.path.exists(f"{folder_daerah_gambar}/6. Subuh/{daerah.lower()}_{waktu_now}_{count}.jpg"):
        count += 1

success = getFrame(sec, count)
initial = count

while success:
    count += 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec, count)

vidcap.release()

try:
    os.remove(f"{video_path}")
    print('berhasil hapus video file')
except:
    pass

berhasil hapus video file
